In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

torch.manual_seed(0)

In [11]:
batch_size = 8
learning_rate = 0.002
num_epochs = 800

In [12]:
normal_dir = 'C:\\Users\\WorkStation\\Desktop\\CAE_dataset\\normal'
abnormal_dir = 'C:\\Users\\WorkStation\\Desktop\\CAE_dataset\\abnormal3'

normal_set = datasets.ImageFolder(
    normal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

abnormal_set = datasets.ImageFolder(
    abnormal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

train_set, test_set = torch.utils.data.random_split(normal_set, [len(normal_set) - 50, 50])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=True)
normal_loader = torch.utils.data.DataLoader(normal_set, batch_size=batch_size, shuffle=True)
abnormal_loader = torch.utils.data.DataLoader(abnormal_set, batch_size=batch_size,shuffle=True)

In [13]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        # Encoder
        self.cnn_layer1 = nn.Sequential(
                        nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                        nn.ReLU(),
                         nn.MaxPool2d(2,2))

        self.cnn_layer2 = nn.Sequential(
                                nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        
        self.cnn_layer3 = nn.Sequential(
                                nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                 nn.MaxPool2d(2,2))

        # Decoder        
        self.tran_cnn_layer1 = nn.Sequential(
                        nn.ConvTranspose2d(64, 32, kernel_size = 2, stride = 2, padding=0),
                        nn.ReLU())
        
        self.tran_cnn_layer2 = nn.Sequential(
                        nn.ConvTranspose2d(32, 16, kernel_size = 2, stride = 2, padding=0),
                        nn.ReLU())

        self.tran_cnn_layer3 = nn.Sequential(
                        nn.ConvTranspose2d(16, 3, kernel_size = 2, stride = 2, padding=0),
                        nn.Sigmoid())
            
            
    def forward(self, x):
        output = self.cnn_layer1(x)
        output = self.cnn_layer2(output)
        output = self.cnn_layer3(output)        
        output = self.tran_cnn_layer1(output)
        output = self.tran_cnn_layer2(output)
        output = self.tran_cnn_layer3(output)

        return output

In [14]:
model = ConvAutoEncoder()
model.load_state_dict(torch.load('best_model_final.pth'))
criterion = nn.MSELoss()

In [15]:
import math

def myEquation4(x):
    
    normal_err = 0
    abnormal_err = 0
    
    for i, (X_normal, _) in enumerate(normal_loader):
        output = model(X_normal)
        loss = criterion(X_normal, output)
        
        if loss.item() > x:
            normal_err += 1
        
    for i, (X_abnormal, _) in enumerate(abnormal_loader):
        output = model(X_abnormal)
        loss = criterion(X_abnormal, output)
        
        if loss.item() < x:
            abnormal_err += 1
      
    err = ((normal_err/len(normal_loader)) + (abnormal_err/len(abnormal_loader)))/2.0
    
    return err

gr = (math.sqrt(5) + 1) / 2

def gss(f, a, b, tol=1e-7):
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(b - a) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c

        # We recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        c = b - (b - a) / gr
        d = a + (b - a) / gr

    return (b + a) / 2

gss(myEquation4, 0.0005, 0.0015)

0.0011706234052549184

In [17]:
myEquation4(0.0011706234052549184)

0.004010695187165776